---

# A Tutorial Implementation of a Dependently Typed Lambda Calculus
### Lucas Dutton
### May 2024
### CAS 781
---

# Prelude: Differences between Haskell and Elm

## Functions and Types

Elm types uses the `type` keyword, and `type alias` for type aliasing.

Haskell uses `data` and `type`.

For this Elm code:
```elm
type Color = Red | Yellow | Green
type alias MaybeInt = Maybe Int
```

we have the corresponding definition in Haskell:

In [1]:
data Color = Red | Yellow | Green
type MaybeInt = Maybe Int

Haskell functions use `::` to denote "type of", as opposed to Elm's `:`.

In [2]:
changeColor :: Color -> Color
changeColor c = case c of
  Red -> Green
  Green -> Yellow
  Yellow -> Red

Instead of using a `case` expression, we can use pattern matching syntax., which also allows wildcards with underscores:

In [3]:
changeColor' :: Color -> Color
changeColor' Red = Green
changeColor' Green = Yellow
changeColor' _ = Red

## Haskell Lists

Recall some basic Elm list operations:
```elm
listInts : List Int
listInts = [3, 5, 7]

consToList : List Int
consToList = 1 :: [3, 5, 7]
```

The equivalent Haskell code is implemented as follows:

In [4]:
listInts :: [Int]
listInts = [3, 5, 7]

consToList :: [Int]
consToList = 1 : [3, 5, 7]

We will make use of a *list indexing operation* `xs !! idx`. This returns the element in `xs` at index `idx`:

In [5]:
consToList !! 2

5

However, indexing out of the list bounds will throw an error!

In [6]:
consToList !! 10

: 

## Do-notation

We define a `Result` time, which is a wrapper around Haskell's `Either` type:

In [7]:
type Result a = Either String a

This says that values are either an error message `String`, or a result type `a`.

We can create `Result a` values and define functions that can throw errors.

Instead of writing `Right x`, we can also write `return x`, i.e. returning a value `x` wrapped in the `Right` constructor:

In [8]:
errorMsg, okValue :: Result Int
errorMsg = Left "an error message"
okValue = Right 3

subtract :: Int -> Result Int
subtract x =
  let minusOneInt = x - 1
  in if minusOneInt < 0 then Left "negative value!" else return minusOneInt

We can chain `subtract`:

In [9]:
minusTwo :: Int -> Result Int
minusTwo x = case subtract x of
  Left err -> Left err
  Right okY -> case subtract okY of
    Left err2 -> Left err2
    Right okZ -> return okZ

A nicer way to write this, which nicely avoids the `case` nesting, is to use `do`-notation:

In [10]:
minusTwoDo :: Int -> Result Int
minusTwoDo x = do
  okY <- subtract x
  okZ <- subtract okY
  return okZ

Line 2: Redundant return
Found:
do okY <- subtract x
   okZ <- subtract okY
   return okZ
Why not:
do okY <- subtract x
   subtract okY

If you read the warning message, you see that we can replace the last two lines with just `subtract okY`! This is because the last line of a do-notation IS the return value (and we don't have to wrap the value in a `return` either):

In [11]:
minusTwoDo' :: Int -> Result Int
minusTwoDo' x = do
  okY <- subtract x
  subtract okY

# Prelude - Some Elm Niceties

## Pipe operators

We define forward and backward piping, which works exactly like their Elm counterparts:

In [12]:
infixr 0 <|
(<|) = ($)

infixl 1 |>
(|>) :: a -> (a -> b) -> b
(|>) a f = f a

## Dict type and operations

We define the `Dict` type, which uses an underlying `Data.Map.Strict`, but works the same way as the Elm `Dict` type:

In [13]:
import qualified Data.Map.Strict as Dict
import qualified Data.Map.Strict as StrictMap (insert)
type Dict = Dict.Map
get = Dict.lookup
insert = StrictMap.insert

In [14]:
emptyDict = Dict.fromList []  -- Also can write Dict.empty
dictWithOneVal = insert "Month" "May" emptyDict
get "Month" dictWithOneVal

Just "May"

In [15]:
get "Day" dictWithOneVal

Nothing

## Error Handling

This imports two functions, `throwError` and `unless`, that work on `Result` types

In [16]:
import Control.Monad.Error.Class (throwError)
import Control.Monad (unless)

In [17]:
exampleError :: Result a
exampleError = throwError "An error!"

In [18]:
exampleError

Left "An error!"

In [19]:
minusTwoGTEOne :: Int -> Result ()
minusTwoGTEOne x = do
  val <- minusTwoDo x
  unless (val >= 1) (throwError "Value is less than one")

In [20]:
minusTwoGTEOne 3

Right ()

In [21]:
minusTwoGTEOne 2

Left "Value is less than one"

Notice that the next function could throw two errors: it is both a negative value, and it is also less than one.

The `Result` type in do-notation always throws the earliest occuring error:

In [22]:
minusTwoGTEOne 1

Left "negative value!"

# Prelude - De Bruijn Indices

Consider the following expression:

```
λ x. λ x. x
```

Which `x` does the body refer to? The outer `x` or the inner `x`?

This isn't easily answered for the lambda calculus; most programming languages have a mechanism for this, called "name shadowing".

The idea of **De Bruijn Indices** is to avoid this problem entirely by using a scoped integer representation of local variables.

Let's look at an example: the λ-term

```
λ x. x (λ y. y x)
```

can be represented as

```
λ. 0 (λ. 0 1)
```

<img style="float:left;border-left:6em solid white" src="./assets/DeBruijnExample.png"/>


- The occurence of the outer `x` is replaced with 0 to signify that `x` is the variable bound by the closest occuring λ binder.
- The inner `y` is bound in a similar fashion
- The second `x` is replaced by 1 to signify that this variable is bounded by two λ's up the syntax tree (indicated by the red arrow)

# Simply Typed Lambda Calculus

The paper introduces the simply-typed lambda calculus, presenting the abstract syntax, evaluation rules and typechecking rules before the Haskell code.

This presentation will introduce the Haskell code concurrently with the mathematical notation.

## Haskell Definitions

### Definition for Names

Variables have 2 different representations:
- They are bounded by a lambda term: We use the `Local` constructor to attach an index to the variable, using De Bruijn representation
- They are free: We refer to them by their *name*, using the `Global` constructor

The combination of using numbers for local variables and names for global variables is called a *locally nameless* representation.

The `Quote` constructor will be used during the process of "quoting", explained later below.

In [23]:
data Name
  = Global String
  | Local  Int
  | Quote  Int
  deriving (Show, Ord, Eq)

### Definition for Types

There are only two constructs for types:

```
τ ::= α        base type
    | τ → τ'   function type
```

In [24]:
data Type
  = TFree Name
  | Fun Type Type
  deriving (Show, Eq)

### Definition for Terms

There are four kinds of terms:
```
e ::= e :: τ    annotated term
   |  x         variable
   |  e e'      function application
   |  λ x → e   lambda abstraction/function declaration
```

The implementation is slightly more complex:
- We separate between *inferrable* terms `TermInfer` and *checkable* terms `TermCheck`
- `TermInfer` contains annotated terms, free/bound variables, and application `:@:`
  - For example, `f a` is written as `f :@: a` in code
- `TermCheck` contains lambda abstractions *without* the bound variable name because of the De Bruijn representation
- `TermCheck` also has *inferrable* terms with the `Inf` constructor

In [25]:
data TermInfer
  = Ann   TermCheck Type
  | Bound Int
  | Free  Name
  | TermInfer :@: TermCheck
  deriving (Show, Eq)
  
data TermCheck
  = Inf  TermInfer
  | Lam  TermCheck
  deriving (Show, Eq)

### Definition for Values

Terms can be *evaluated* to values:
```
v ::= λ x −> v  lambda abstraction
   |  n         annotated term 
   
n ::= x         variable
   |  n v       application
```

A value is either a lambda abstraction or a *neutral term*, i.e. a variable applied to a (possibly empty) sequence of values

Notice that `VLam` takes a Haskell function as an argument. This prevents us from printing `VLam`! We will define quotation to deal with this issue.

In [26]:
data Value
  = VLam    (Value -> Value)
  | VNeutral Neutral

data Neutral
  = NFree Name
  | NApp Neutral Value

In [27]:
vfree :: Name -> Value
vfree n = VNeutral (NFree n)

## Evaluation

To evaluate *bound* variables, we need a way of looking them up in an environment.

The environment is defined as a list, but is still defined as as mapping: We map the De Bruijn indices based on their position in the list!

In [28]:
type Env = [Value]

Back to the previous example:

<img style="float:left;border-left:6em solid white" src="./assets/DeBruijnExample.png"/>

- We start with the empty environment `env = []`
- First, we insert the value of `x` into the empty environment: `x :: []`
- To look up the value of `x` in the outer lambda, we index at the 0th position: `env !! 0`
- When entering the inner lambda, we put the value of `y` at the front of the list: `y :: [x]`. The list is now `[y, x]`.
- Now looking up `x` is done by `env !! 1`, and `y` is `env !! 0`

<img style="float:left;border-left:6em solid white" src="./assets/simple-eval.jpg"/>

Evaluation is given in the above figure. The implementation splits the rules into `evalInfer` and `evalCheck`, for inferred terms and checked terms.

The notation `e ⇓ v` means the result of *completely* evaluating `e` is `v` - this uses big-step semantics.

1. Annotations get ignored - we just evaluate the expression `e`.
2. Variables evaluate to themselves.
3. Applications can be reduced to two subcases:
   1. If `e` evaluates to a lambda, we can perform substitution on occurences of `x` with `e'` in `v`, yielding the final term `v'`
   2. If `e` evaluates to a neutral term, we cannot proceed further! We construct a new neutral term from the results of evaluating the two subterms.
4. For lambdas themselves, we directly evaluate the body.

In [29]:
evalInfer :: TermInfer -> Env -> Value
evalInfer (Ann e _)  d = evalCheck e d
evalInfer (Free x)   d = vfree x
evalInfer (Bound i)  d = d !! i
evalInfer (e :@: e') d = vapp (evalInfer e d) (evalCheck e' d)

evalCheck :: TermCheck -> Env -> Value
evalCheck (Inf  i) d = evalInfer i d
evalCheck (Lam  e) d = VLam (\x -> evalCheck e (x : d))

vapp :: Value -> Value -> Value
vapp (VLam f)     v = f v
vapp (VNeutral n) v = VNeutral (NApp n v)

The evaluation returns `Values` in the Haskell code, and contains further details:
- Evaluating `Free` returns the name of the variable itself, using the `vfree` smart constructor.
- Evaluating `Bound` uses the indexing function.
- Evaluating `:@:` calls `evalInfer` on the function term, and `evalCheck` on the argument term.
  - `vapp` returns `f v`, if the function term is a lambda value `VLam`
  - It returns `VNeutral` otherwise, which wraps an application value `NApp`
- Evaluating `Lam` creates a *Haskell* function that takes a value `x` as an argument, and inserts `x` into the environment.

## Contexts

Contexts can be either of the following:

```
Γ ::= ε         empty context 
  |  Γ, α :: *  adding a type identifier
  |  Γ, x :: τ  adding a term identifier
```

We store both terms and types in the context. 
- A term `x :: τ` reads `x` has type `τ`
- A type `α :: *` reads `α` has kind `*`

In [30]:
data Kind = TypeKind
  deriving (Show)

data Info
  = HasKind Kind
  | HasType Type
  deriving (Show)

<img style="float:left;border-left:6em solid white" src="./assets/simple-context.jpg"/>

Valid contexts are defined with judgment rules.
- The first three rules correspond to each constructor of the context. Note that for terms to be valid, their types must be kinded with `TypeKind`
- The `TVAR` rule states that a type is well-formed when it is kinded with `TypeKind` in the context
- `FUN` states that for a function arrow to be valid, `τ` and `τ'` must be kinded with `TypeKind`

We assume that all contexts are valid from here out. **NOTE: When creating examples, we must ensure our contexts are valid!**

The `Context` is implemented as a dictionary mapping `Names` to their `Info`

In [31]:
type Context = Dict Name Info

In [32]:
kindCheck :: Context -> Type -> Kind -> Result ()
kindCheck ctx (TFree x) TypeKind = 
  case get x ctx of
    Just (HasKind TypeKind) -> return ()
    Nothing                 -> throwError "unknown identifier"
kindCheck ctx (Fun k1 k2) TypeKind = do
  kindCheck ctx k1 TypeKind
  kindCheck ctx k2 TypeKind

## Type Checking

### Substitution

Substitutions deal only with bound variables (free variables are left unchanged).

`substInfer` and `substCheck` takes term `r` and attempts to substitute.

The two interesting cases to look at are for `Bound k` and `Lam e`:
- We check the index `i`, and if it is equal to the the bound variable index `j`, we return the substitution term `r`. Otherwise, leave the bound variable unchanged.
- For `Lam` terms, we increment the index `i` and proceed into the body `e`.

Substitution will be used in the type checking procedure.

In [33]:
substInfer :: Int -> TermInfer -> TermInfer -> TermInfer
substInfer i r (Ann e t)   = Ann (substCheck i r e) t
substInfer i r (Bound j)   = if i == j then r else Bound j
substInfer i r (Free y)    = Free y
substInfer i r (e1 :@: e2) = substInfer i r e1 :@: substCheck i r e2

substCheck :: Int -> TermInfer -> TermCheck -> TermCheck
substCheck i r (Inf e)  = Inf (substInfer i r e)
substCheck i r (Lam e)  = Lam (substCheck (i + 1) r e)

As with bidirectional type checking, we split into "infer mode" `typeInfer` and "check mode" `typeCheck`. The top-level function is `typeInfer0`.

<img style="float:left;border-left:2em solid white" src="./assets/simple-typerules.jpg"/>

For inferrable terms:
- `Ann e t` `t` to ensure it is `TypeKind`, and then `typeChecks` the term with its annotated type `t`
- `Free x` looks up the type of `x` in the context
- `e1 :@: e2` infers the function arrow type for `e1`, then uses the return type `t2` to check `e2`

For checkable terms:
- `Inf e` switches from check mode to infer mode on term `e`. The inferred type `t'` must equal the type `t` it is checking on
- `Lam e` is complicated:
  - The checked type must be a function type `Fun t1 t2`.
  - We go into one additional nesting level. So the index is incremented `i + 1`.
  - The local variable `i` is inserted into the environment `(ctx |> insert (Local i) (HasType t1))`. Since it is the input type, it has type `t1`.
  - We substitute all bound occurences of `i` in `e` with `Free (Local i)`, i.e. making it a free variable. Consequently, `typeInfer` will never have a case for bound variables!

In [34]:
typeInfer0 :: Context -> TermInfer -> Result Type
typeInfer0 ctx term = typeInfer 0 ctx term

typeInfer :: Int -> Context -> TermInfer -> Result Type
typeInfer i ctx (Ann e t) = do
  kindCheck ctx t TypeKind
  typeCheck i ctx e t
  return t
typeInfer i ctx (Free x) = 
  case get x ctx of
    Just (HasType t) -> return t
    Nothing          -> throwError "typeInfer Free case: unknown identifier"
typeInfer i ctx (e1 :@: e2) = do
  tyFun <- typeInfer i ctx e1 
  case tyFun of
    Fun t1 t2 -> do
      typeCheck i ctx e2 t1
      return t2
    _ -> throwError "typeInfer :@: case: illegal application"
    
typeCheck :: Int -> Context -> TermCheck -> Type -> Result ()
typeCheck i ctx (Inf e) t = do
  t' <- typeInfer i ctx e
  unless (t == t') (throwError "typeCheck Inf case: type mismatch")
typeCheck i ctx (Lam e) (Fun t1 t2) =
  typeCheck (i + 1) (ctx |> insert (Local i) (HasType t1)) 
            (substCheck 0 (Free (Local i)) e)
            t2
typeCheck i ctx _ _ = throwError "typeCheck non-case: type mismatch"

Line 2: Eta reduce
Found:
typeInfer0 ctx term = typeInfer 0 ctx term
Why not:
typeInfer0 = typeInfer 0

## Quotation

Quotation converts `Value`s into `Term`s. Recall the problem with values are for lambda values `VLam (Value -> Value)`, which does not allow comparisons and printing to the console.

There are two interesting cases for quotation:
- `VLam f` increments the bound index `(i + 1)`, and applies `f` to the the bound variable `i`, which we wrap with the `Quote` constructor. We recursively call `quote` on the body.
- Variable `NFree x` is checked with `boundFree`:
  - If `x` is a `Quote`, it means it is a bound variable. The computation `(i - k - 1)` restores the bound variable level.
  - Otherwise, it is a free variable, which we wrap with `Free`

In [35]:
quote0 :: Value -> TermCheck
quote0 = quote 0

quote :: Int -> Value -> TermCheck
quote i (VLam f)     = Lam <| quote (i + 1) (f (vfree <| Quote i))
quote i (VNeutral n) = Inf (neutralQuote i n)

neutralQuote :: Int -> Neutral -> TermInfer
neutralQuote i (NFree x)  = boundfree i x
neutralQuote i (NApp n v) = neutralQuote i n :@: quote i v

boundfree :: Int -> Name -> TermInfer
boundfree i (Quote k) = Bound (i - k - 1)
boundfree i x         = Free x

The calculation below shows how `quote` is computed on the value `(VLam (\x -> VLam (\y -> x)))`

```haskell
    quote 0 (VLam (\x -> VLam (\y -> x)))
  =⟨ `quote` for VLam ⟩
    Lam <| quote 1 <| (\x -> VLam (\y -> x)) (vfree <| Quote 0)
  =⟨ function application ⟩
    Lam <| quote 1 <| VLam (\y -> (vfree <| Quote 0))
  =⟨ `quote` for VLam ⟩
    Lam <| Lam <| quote 2 (\y -> (vfree <| Quote 0)) (vfree <| Quote 0)
  =⟨ function application ⟩
    Lam <| Lam <| quote 2 ((vfree <| Quote 0))
  =⟨ definition of `vfree` ⟩
    Lam <| Lam <| quote 2 (VNeutral (NFree (Quote 0)))
  =⟨ `quote` for VLam ⟩
    Lam <| Lam <| Inf (neutralQuote 2 (NFree (Quote 0)))
  =⟨ `neutralQuote` for NFree ⟩
    Lam <| Lam <| Inf (boundFree 2 (Quote 0))
  =⟨ `neutralQuote` for Quote ⟩
    Lam <| Lam <| Inf (Bound (2 - 0 - 1))
  =⟨ arithmetic ⟩
    Lam (Lam (Inf (Bound 1)))
```

In [36]:
quote 0 (VLam (\x -> VLam (\y -> x)))

Line 1: Use const
Found:
\ y -> x
Why not:
const x

Lam (Lam (Inf (Bound 1)))

## Examples

In [37]:
id'    = Lam (Inf (Bound 0))
const' = Lam (Lam (Inf (Bound 1)))

tfree a = TFree (Global a)
free x  = Inf (Free (Global x))

term1 = Ann id' (Fun (tfree "a") (tfree "a"))  :@:  free "y"
term2 = Ann const' (Fun (Fun (tfree "b") (tfree "b"))
                        (Fun (tfree "a")
                              (Fun (tfree "b") (tfree "b"))))
        :@: id' :@: free "y"
env1 = [(Global "y", HasType (tfree "a")),
        (Global "a", HasKind TypeKind)] |> Dict.fromList
env2 = [(Global "b", HasKind TypeKind)] ++ (Dict.toList env1) |> Dict.fromList

Line 14: Use :
Found:
[(Global "b", HasKind TypeKind)]
  ++ (Dict.toList env1) |> Dict.fromList
Why not:
(Global "b", HasKind TypeKind)
  : (Dict.toList env1) |> Dict.fromListLine 14: Redundant bracket
Found:
(Dict.toList env1) |> Dict.fromList
Why not:
Dict.toList env1 |> Dict.fromList

In [38]:
quote0 (evalInfer term1 [])

Inf (Free (Global "y"))

In [39]:
quote0 (evalInfer term2 [])

Lam (Inf (Bound 0))

In [40]:
typeInfer0 env1 term1

Right (TFree (Global "a"))

In [41]:
typeInfer0 env2 term2

Right (Fun (TFree (Global "b")) (TFree (Global "b")))

# Dependent Types

## Polymorphic Functions

In Haskell (and Elm), we can define polymorphic functions. For instance, the identity function that returns the input value itself:

In [42]:
myIdentity :: a -> a
myIdentity x = x

There is an implicit quantifier on `a` here - we can write this mathematically:
```haskell
id :: ∀ α. α -> α
id = λ x -> x
```

We can make polymorphic calls explicit by instantiating the type. Hence, the identity function takes *two* arguments: a type for `α` and a value of type `α`:

```haskell
id :: ∀ α. α -> α
id = λ (α :: *) (x :: α) -> x

id Bool True :: Bool
id Int 3 :: Int
```

## Dependent Function Space

In languages like C and Java, we can declare arrays with an associated length. For example `int[3]` is an array of integers of length 3.

Notice the number 3 is part of the type declaration. Say we want to do the same for Haskell lists (called vectors, or `Vec`):

```haskell
intList3 :: Vec 3 Int
intList3 = [3, 5, 7]
```

Unfortunately, the number 3 is a value. It is not a type!

Let's allow any value to be types for the moment. We can then, for instance, declare the identity function over vec:

```haskell
∀ α :: * .∀ n :: Nat.∀ v :: Vec α n. Vec α n
```

This term `∀ x :: ρ. ρ'` is called the *dependent function space*. It generalizes from `ρ → ρ'`

Since the `v` is redundant in `Vec` above, we can use the ordinary function arrow instead, and rewrite as follows:

```haskell
∀ α :: * .∀ n :: Nat.Vec α n → Vec α n
```

## Definitions

### Terms

Since types can be any value, we do not need a separate type for type-level constructs and term-level constructs. The only difference in `TermInfer` is the introduction of `Pi` for the dependent function space.

The abstract syntax is given as follows. Note that `ρ` and `κ` refer to terms that play the role of types or kinds respectively
```
e, ρ, κ ::= e :: ρ        annotated term
         |  *             the type of types
         |  ∀ x :: ρ. ρ'  dependent function space
         |  x             variable
         |  e e'          application
         |  λ x → e       lambda abstraction
```         

In [43]:
data TermInfer =
    Ann TermCheck TermCheck
  | TypeKind
  | Pi TermCheck TermCheck
  | Bound Int
  | Free Name
  | TermInfer :@: TermCheck
  deriving (Show, Eq)
  
data TermCheck
  = Inf  TermInfer
  | Lam  TermCheck
  deriving (Show, Eq)

### Values

Two additional constructors must be added to `Value` since types are also terms:
- `VTypeKind`, indicating terms that are kinded as a type
- `VPi`, indicating a ∀ term. The second argument is `Value -> Value`, similar to `VLam`

The definition follows the abstract syntax:
```
v, τ ::= n             neutral term
      |  *             the type of types
      |  ∀ x :: τ. τ'  dependent function space
      |  λ x → v       lambda abstraction
```

In [44]:
data Value =
    VLam (Value -> Value)
  | VTypeKind
  | VPi Value (Value -> Value)
  | VNeutral Neutral

data Neutral
  = NFree Name
  | NApp Neutral Value
  
vfree :: Name -> Value
vfree n = VNeutral (NFree n)

## Evaluation

Substitution remains almost the same - but we have additional cases for `TypeKind` and `Pi`:

In [45]:
substInfer :: Int -> TermInfer -> TermInfer -> TermInfer
substInfer i r (Ann e t)   = Ann (substCheck i r e) t
substInfer i r (Bound j)   = if i == j then r else Bound j
substInfer i r (Free y)    = Free y
substInfer i r (e1 :@: e2) = substInfer i r e1 :@: substCheck i r e2
-- new cases
substInfer i r TypeKind    = TypeKind
substInfer i r (Pi t1 t2)  = Pi (substCheck i r t1) (substCheck (i + 1) r t2)

substCheck :: Int -> TermInfer -> TermCheck -> TermCheck
substCheck i r (Inf e)  = Inf (substInfer i r e)
substCheck i r (Lam e)  = Lam (substCheck (i + 1) r e)

vapp :: Value -> Value -> Value
vapp (VLam f)     v = f v
vapp (VNeutral n) v = VNeutral (NApp n v)

Evaluation is also the same, with two additional cases for `TypeKind` and `Pi`.

The judgment rules also reflect these additions:

<img style="float:left;border-left:6em solid white" src="./assets/dep-eval.jpg"/>

In [46]:
type Env = [Value]

evalInfer :: TermInfer -> Env -> Value
evalInfer (Ann e _)  d = evalCheck e d
evalInfer (Free x)   d = vfree x
evalInfer (Bound i)  d = d !! i
evalInfer (e :@: e') d = vapp (evalInfer e d) (evalCheck e' d)
-- new cases
evalInfer TypeKind   d = VTypeKind
evalInfer (Pi t1 t2) d = VPi (evalCheck t1 d) (\x -> evalCheck t2 (x : d))

evalCheck :: TermCheck -> Env -> Value
evalCheck (Inf  i) d = evalInfer i d
evalCheck (Lam  e) d = VLam (\x -> evalCheck e (x : d))

## Quotation

`quote` has two additional cases also for `VPi` and `VTypeKind`

In [47]:
quote0 :: Value -> TermCheck
quote0 = quote 0

quote :: Int -> Value -> TermCheck
quote i (VLam f)     = Lam <| quote (i + 1) (f (vfree <| Quote i))
quote i (VNeutral n) = Inf (neutralQuote i n)
quote i VTypeKind    = Inf TypeKind
quote i (VPi v f)    = Inf (Pi (quote i v) (quote (i + 1) (f (vfree (Quote i)))))

neutralQuote :: Int -> Neutral -> TermInfer
neutralQuote i (NFree x)  = boundfree i x
neutralQuote i (NApp n v) = neutralQuote i n :@: quote i v

boundfree :: Int -> Name -> TermInfer
boundfree i (Quote k) = Bound (i - k - 1)
boundfree i x         = Free x

In [48]:
quote 0 (VLam (\x -> VLam (\y -> x)))

Line 1: Use const
Found:
\ y -> x
Why not:
const x

Lam (Lam (Inf (Bound 1)))

## Contexts

Since `Types` are now `Values`, we can then alias them accordingly. This also simplifies the `Context` to map `Names` to `Types` directly.

<img style="float:left;border-left:6em solid white" src="./assets/dep-ctx.jpg"/>

In [49]:
type Type    = Value
type Context = Dict Name Type

## Type Checking

<img style="float:left;border-left:2em solid white" src="./assets/dep-tycheck.jpg"/>

Type checking introduces two new cases for `TypeKind` and `Pi`, and modifications to `Ann`, `App` and `Lam`. The biggest change to the rules involve evaluation; certain terms require evaluating subterms.

- The `TypeKind` term resolves to itself.
- Dependent function space `Pi` requires some work:
  1. The bound term `p` is typechecked to ensure it is of `TypeKind`
  2. Then, `p` is evaluated down to a type value `τ`
  3. The type body `p'` is then typechecked to ensure it is also of `TypeKind`, after inserting `x :: τ` into the environment
  
Modifications include the following:
- `Ann` checks that the annotation `p` is `TypeKind`. Then we evaluate `p` to `τ` and use it to check the term `e`
- `App` now checks `e` to be the dependent function space instead of the original function arrow
- `Lam` now expects the dependent function space instead of the function arrow

In [50]:
typeInfer0 :: Context -> TermInfer -> Result Type
typeInfer0 ctx term = typeInfer 0 ctx term

typeInfer :: Int -> Context -> TermInfer -> Result Type
typeInfer i ctx (Ann e p) = do
  typeCheck i ctx p VTypeKind
  let t = evalCheck p []
  typeCheck i ctx e t
  return t
typeInfer i ctx TypeKind = return VTypeKind
typeInfer i ctx (Pi p1 p2) = do
  typeCheck i ctx p1 VTypeKind
  let t = evalCheck p1 []
  typeCheck (i + 1) (ctx |> insert (Local i) t) (substCheck 0 (Free (Local i)) p2) VTypeKind
  return VTypeKind
typeInfer i ctx (Free x) = 
  case get x ctx of
    Just t           -> return t
    Nothing          -> throwError "typeInfer Free case: unknown identifier"
typeInfer i ctx (e1 :@: e2) = do
  vDepPair <- typeInfer i ctx e1 
  case vDepPair of
    VPi t1 t2 -> do
      typeCheck i ctx e2 t1
      return (t2 <| evalCheck e2 [])
    _ -> throwError "typeInfer :@: case: illegal application"
    
typeCheck :: Int -> Context -> TermCheck -> Type -> Result ()
typeCheck i ctx (Inf e) v = do
  v' <- typeInfer i ctx e 
  unless (quote0 v == quote0 v') (throwError "typeCheck Inf case: type mismatch")
typeCheck i ctx (Lam e) (VPi t1 t2) =
  typeCheck (i + 1) (ctx |> insert (Local i) t1)
            (substCheck 0 (Free (Local i)) e) (t2 (vfree (Local i)))
typeCheck i ctx _ _ = throwError "typeCheck non-case: type mismatch"

Line 2: Eta reduce
Found:
typeInfer0 ctx term = typeInfer 0 ctx term
Why not:
typeInfer0 = typeInfer 0

## Examples

In [51]:
id' = Lam (Lam (Inf <| Bound 0))
free x  = Inf (Free (Global x))
tfree x = Free (Global x)


-- ∀ x :: *. ∀ 
typeId = Inf (Pi (Inf TypeKind) (Inf <| Pi (Inf <| Bound 0) (Inf <| Bound 1)))
termId = Ann id' typeId

In [52]:
quote0 <| evalCheck typeId []

Inf (Pi (Inf TypeKind) (Inf (Pi (Inf (Bound 0)) (Inf (Bound 1)))))

In [53]:
quote0 (evalInfer termId [])

Lam (Lam (Inf (Bound 0)))

In [54]:
boolType = Inf <| Ann (free "Bool") (Inf TypeKind)
falseTerm = Inf <| Ann (free "False") (free "Bool")

In [55]:
fakeType = Inf <| Ann (free "Fake") (Inf TypeKind)
fakeTerm = Inf <| Ann (free "FakeTerm") (free "Fake")

In [56]:
quote0 (evalInfer (termId :@: boolType) [])

Lam (Inf (Bound 0))

In [57]:
quote0 (evalInfer (termId :@: boolType :@: falseTerm) [])

Inf (Free (Global "False"))

In [58]:
quote0 (evalInfer (termId :@: boolType :@: fakeTerm) [])

Inf (Free (Global "FakeTerm"))

In [59]:
ctx = 
  Dict.fromList <|
     [(Global "Bool", VTypeKind), (Global "False", vfree <| Global "Bool")] 
  ++ [(Global "Fake", VTypeKind), (Global "FakeTerm", vfree <| Global "Fake")]
runInfer = do
  res <- typeInfer0 ctx (termId :@: boolType :@: fakeTerm)
  pure $ quote0 res
runInfer

Left "typeCheck Inf case: type mismatch"

# Dependent Data Types: Natural Numbers

The syntax can be extended with natural numbers.

In Haskell, we could define the `Nat` data type and define addition over natural numbers:

In [ ]:
data Nat = Zero | Succ Nat

plus :: Nat -> Nat -> Nat
plus Zero n      = n
plus (Succ k) n  = Succ (plus k n)

Our dependently typed language cannot dop pattern patches or make recursive calls. One way around it is to define an *eliminator*.

Natural numbers can be *folded* using the following type:

```haskell
foldNat :: ∀α :: *.α → (α → α) → Nat → α
```

The eliminator makes clearer what `α` should be: we use `m :: Nat -> *`:

```haskell
natElim :: ∀m :: Nat → *.    m Zero
                     → (∀l :: Nat.m l → m (Succ l))
                     → ∀k :: Nat. m k
```

## Definitions

### Terms

Four new constructors are introduced. Note that `Succ` and `NatElim` all take checkable terms for their arguments.

In [60]:
data TermInfer =
    Ann TermCheck TermCheck
  | TypeKind
  | Pi TermCheck TermCheck
  | Bound Int
  | Free Name
  | TermInfer :@: TermCheck
  | Nat
  | NatElim TermCheck TermCheck TermCheck TermCheck
  | Zero
  | Succ TermCheck
  deriving (Show, Eq)
  
data TermCheck
  = Inf  TermInfer
  | Lam  TermCheck
  deriving (Show, Eq)

### Values

The nat constructors have corresponding values; we have `NNatElim` as a neutral value, as there will be cases where the eliminator for natural numbers can be stuck when the number being eliminated does not evaluate to a constructor.

In [61]:
data Value =
    VLam (Value -> Value)
  | VTypeKind
  | VPi Value (Value -> Value)
  | VNeutral Neutral
  | VNat
  | VZero
  | VSucc Value

data Neutral
  = NFree Name
  | NApp Neutral Value
  | NNatElim Value Value Value Neutral
  
vfree :: Name -> Value
vfree n = VNeutral (NFree n)

## Evaluation

In [62]:
substInfer :: Int -> TermInfer -> TermInfer -> TermInfer
substInfer i r (Ann e t)   = Ann (substCheck i r e) t
substInfer i r (Bound j)   = if i == j then r else Bound j
substInfer i r (Free y)    = Free y
substInfer i r (e1 :@: e2) = substInfer i r e1 :@: substCheck i r e2
substInfer i r TypeKind    = TypeKind
substInfer i r (Pi t1 t2)  = Pi (substCheck i r t1) (substCheck (i + 1) r t2)
substInfer i r Nat         = Nat
substInfer i r Zero        = Zero
substInfer i r (Succ n)    = Succ (substCheck i r n)
substInfer i r (NatElim m mz ms n) =
  NatElim (substCheck i r m) (substCheck i r mz) (substCheck i r ms) (substCheck i r n)

substCheck :: Int -> TermInfer -> TermCheck -> TermCheck
substCheck i r (Inf e)  = Inf (substInfer i r e)
substCheck i r (Lam e)  = Lam (substCheck (i + 1) r e)

vapp :: Value -> Value -> Value
vapp (VLam f)     v = f v
vapp (VNeutral n) v = VNeutral (NApp n v)

<img style="float:left;border-left:6em solid white" src="./assets/nat-eval.jpg"/>

The first three cases of evaluating natural numbers are straightforward. For the eliminator:
- If `k` evaluates to `Zero`, then the evaluation of `mz` is returned.
- If `k` evaluates to a `Succ l`, the evaluation of `ms l (natElim m mz ms l)` is returned. Note that the inner `natElim` is recursive.
- Otherwise, the evaluation is "stuck", and `k` is evaluated to a neutral term.

In [63]:
type Env = [Value]

evalInfer :: TermInfer -> Env -> Value
evalInfer (Ann e _)  d = evalCheck e d
evalInfer (Free x)   d = vfree x
evalInfer (Bound i)  d = d !! i
evalInfer (e :@: e') d = vapp (evalInfer e d) (evalCheck e' d)
-- new cases
evalInfer TypeKind   d = VTypeKind
evalInfer (Pi t1 t2) d = VPi (evalCheck t1 d) (\x -> evalCheck t2 (x : d))
evalInfer Nat        d = VNat
evalInfer Zero       d = VZero
evalInfer (Succ k)   d = VSucc (evalCheck k d)
evalInfer (NatElim m mz ms k) d =
  let mzVal = evalCheck mz d
      msVal = evalCheck ms d
      rec kVal = case kVal of
        VZero      -> mzVal
        VSucc l    -> msVal `vapp` l `vapp` rec l
        VNeutral k -> VNeutral (NNatElim (evalCheck m d) mzVal msVal k)
        _          -> error "internal: eval natElim"
  in rec (evalCheck k d)

evalCheck :: TermCheck -> Env -> Value
evalCheck (Inf  i) d = evalInfer i d
evalCheck (Lam  e) d = VLam (\x -> evalCheck e (x : d))

## Quotation

In [64]:
quote0 :: Value -> TermCheck
quote0 = quote 0

quote :: Int -> Value -> TermCheck
quote i (VLam f)     = Lam <| quote (i + 1) (f (vfree <| Quote i))
quote i (VNeutral n) = Inf (neutralQuote i n)
quote i VTypeKind    = Inf TypeKind
quote i (VPi v f)    = Inf (Pi (quote i v) (quote (i + 1) (f (vfree (Quote i)))))
quote i VNat         = Inf Nat
quote i VZero        = Inf Zero
quote i (VSucc n)    = Inf <| Succ (quote i n)

neutralQuote :: Int -> Neutral -> TermInfer
neutralQuote i (NFree x)  = boundfree i x
neutralQuote i (NApp n v) = neutralQuote i n :@: quote i v
neutralQuote i (NNatElim m z s n) =
  NatElim (quote i m) (quote i z) (quote i s) (Inf <| neutralQuote i n)
  
boundfree :: Int -> Name -> TermInfer
boundfree i (Quote k) = Bound (i - k - 1)
boundfree i x         = Free x

## Contexts (unchanged)

In [65]:
type Type    = Value
type Context = Dict Name Type

## Type Checking

<img style="float:left;border-left:6em solid white" src="./assets/nat-typerules.jpg"/>

Again, the first three rules are straightforward. For `natElim`:
- Typecheck `m` to ensure its type is `m → *`
- Evaluate `m Zero`, obtaining `τ`
- Typecheck `mz` against `τ`
- Evaluate the type of `ms` and typecheck it. In the implementation, we skip the evaluation step and provide the type directly
- The type is then `m k`, where `k` is first typechecked to be a `Nat`

In [66]:
typeInfer0 :: Context -> TermInfer -> Result Type
typeInfer0 ctx term = typeInfer 0 ctx term

typeInfer :: Int -> Context -> TermInfer -> Result Type
typeInfer i ctx (Ann e p) = do
  typeCheck i ctx p VTypeKind
  let t = evalCheck p []
  typeCheck i ctx e t
  return t
typeInfer i ctx TypeKind = return VTypeKind
typeInfer i ctx (Pi p1 p2) = do
  typeCheck i ctx p1 VTypeKind
  let t = evalCheck p1 []
  typeCheck (i + 1) (ctx |> insert (Local i) t) (substCheck 0 (Free (Local i)) p2) VTypeKind
  return VTypeKind
typeInfer i ctx (Free x) = 
  case get x ctx of
    Just t           -> return t
    Nothing          -> throwError "typeInfer Free case: unknown identifier"
typeInfer i ctx (e1 :@: e2) = do
  vDepPair <- typeInfer i ctx e1 
  case vDepPair of
    VPi t1 t2 -> do
      typeCheck i ctx e2 t1
      return (t2 <| evalCheck e2 [])
    _ -> throwError "typeInfer :@: case: illegal application"
typeInfer i ctx Nat      = return VTypeKind
typeInfer i ctx Zero     = return VNat
typeInfer i ctx (Succ k) = do
  typeCheck i ctx k VNat
  return VNat
typeInfer i ctx (NatElim m mz ms k) = do
  typeCheck i ctx m (VPi VNat (const VTypeKind))
  let mVal = evalCheck m []
  typeCheck i ctx mz (mVal `vapp` VZero)
  typeCheck i ctx ms (VPi VNat (\l -> VPi (mVal `vapp` l) (\_ -> mVal `vapp` VSucc l)))
  typeCheck i ctx k VNat
  let kVal = evalCheck k []
  return (mVal `vapp` kVal)
    
typeCheck :: Int -> Context -> TermCheck -> Type -> Result ()
typeCheck i ctx (Inf e) v = do
  v' <- typeInfer i ctx e 
  unless (quote0 v == quote0 v') (throwError "typeCheck Inf case: type mismatch")
typeCheck i ctx (Lam e) (VPi t1 t2) =
  typeCheck (i + 1) (ctx |> insert (Local i) t1)
            (substCheck 0 (Free (Local i)) e) (t2 (vfree (Local i)))
typeCheck i ctx _ _ = throwError "typeCheck non-case: type mismatch"

Line 2: Eta reduce
Found:
typeInfer0 ctx term = typeInfer 0 ctx term
Why not:
typeInfer0 = typeInfer 0

## Examples

In [67]:
zeroTy = VNat
succTy = VPi VNat (\ _ -> VNat)
natTy = VTypeKind
natElimTy = 
  VPi (VPi VNat (\ _ -> VTypeKind)) (\ m ->
                               VPi (m `vapp` VZero) (\ _ ->
                               VPi (VPi VNat (\ k -> VPi (m `vapp` k) (\ _ -> (m `vapp` (VSucc k))))) ( \ _ ->
                               VPi VNat (\ n -> m `vapp` n))))
                               
ctx = 
  Dict.fromList <|
     [(Global "Zero", zeroTy),
             (Global "Succ", succTy),
             (Global "Nat", natTy),
             (Global "natElim", natElimTy)]

zeroVal = VZero
succVal = VLam (\n -> VSucc n)
natVal = VNat
natElimVal = Lam <| Lam <| Lam <| Lam <| Inf <| NatElim (Inf (Bound 3)) (Inf (Bound 2)) (Inf (Bound 1)) (Inf (Bound 0))

plus = (Ann natElimVal <| quote0 natElimTy) :@: (Lam <| Inf <| Pi (Inf Nat) (Inf Nat)) :@: (Lam <| Inf (Bound 0)) :@: (Lam <| Lam <| Lam <| Inf <| Succ <| Inf (Bound 1 :@: Inf (Bound 0)))

Line 2: Use const
Found:
\ _ -> VNat
Why not:
const VNatLine 5: Use const
Found:
\ _ -> VTypeKind
Why not:
const VTypeKindLine 7: Use const
Found:
\ _ -> (m `vapp` (VSucc k))
Why not:
const (m `vapp` (VSucc k))Line 7: Redundant bracket
Found:
\ _ -> (m `vapp` (VSucc k))
Why not:
\ _ -> m `vapp` (VSucc k)Line 7: Redundant bracket
Found:
m `vapp` (VSucc k)
Why not:
m `vapp` VSucc kLine 18: Avoid lambda
Found:
(\ n -> VSucc n)
Why not:
VSucc

In [68]:
two = Succ <| Inf <| Succ <| Inf Zero
twoPlusTwo = plus :@: Inf two :@: Inf two
twoPlusTwo

((((Ann (Lam (Lam (Lam (Lam (Inf (NatElim (Inf (Bound 3)) (Inf (Bound 2)) (Inf (Bound 1)) (Inf (Bound 0)))))))) (Inf (Pi (Inf (Pi (Inf Nat) (Inf TypeKind))) (Inf (Pi (Inf (Bound 0 :@: Inf Zero)) (Inf (Pi (Inf (Pi (Inf Nat) (Inf (Pi (Inf (Bound 2 :@: Inf (Bound 0))) (Inf (Bound 3 :@: Inf (Succ (Inf (Bound 1))))))))) (Inf (Pi (Inf Nat) (Inf (Bound 3 :@: Inf (Bound 0))))))))))) :@: Lam (Inf (Pi (Inf Nat) (Inf Nat)))) :@: Lam (Inf (Bound 0))) :@: Lam (Lam (Lam (Inf (Succ (Inf (Bound 1 :@: Inf (Bound 0)))))))) :@: Inf (Succ (Inf (Succ (Inf Zero))))) :@: Inf (Succ (Inf (Succ (Inf Zero))))

In [69]:
quote0 (evalInfer twoPlusTwo [])

Inf (Succ (Inf (Succ (Inf (Succ (Inf (Succ (Inf Zero))))))))

In [70]:
runInferTwoPlusTwo = do
  res <- typeInfer0 ctx (twoPlusTwo)
  pure $ quote0 res
runInferTwoPlusTwo

Line 2: Redundant bracket
Found:
(twoPlusTwo)
Why not:
twoPlusTwo

Right (Inf Nat)

# References

[1] Löh, Andres, Conor McBride, and Wouter Swierstra. "A tutorial implementation of a dependently typed lambda calculus." Fundamenta informaticae 102.2 (2010): 177-207.